In [39]:
from elasticsearch import Elasticsearch
from retrievals import baseline_retrieval, advanced_method
from baseline_retrieval import bulk_index, get_queries, get_relevance_scores

DATA_FILE = "../data/collection.tsv"

# Query documents. Contains query id and query text apears
QUERIES_TRAIN = "../data/queries.train.tsv"
QUERIES_EVAL = "../data/queries.eval.tsv"
QUERIES_DEV = "../data/queries.dev.tsv"
QUERY_FILES = [QUERIES_DEV, QUERIES_TRAIN, QUERIES_EVAL]

INDEX_SETTINGS = {
    "mappings": {
        "properties": {
            "body": {
                "type": "text",
                "term_vector": "with_positions",
                "analyzer": "english",
            },
        }
    }
}

# Evaluation scores
RELEVANCE_SCORES = "../data/2019qrels-pass.txt"

INDEX_NAME = "dev_index"

In [40]:
# Index some documents
n = 100  # number of documents to index
es = Elasticsearch()    
es.info()

bulk_index(
    es, data_file=DATA_FILE, index=INDEX_NAME, index_settings=INDEX_SETTINGS, cutoff=n
)

In [41]:
queries = get_queries()
rel_scores = get_relevance_scores()

In [42]:
id_query = [id_and_query for id_and_query in queries.items()]

query_id, query = id_query[910884]
# query has / in it
query = query.replace('/', '')

print(query_id, query, sep='|')

527433|types of dysarthria from cerebral palsy


In [43]:
baseline = baseline_retrieval(es, INDEX_NAME, query, 10)
print(baseline)

['15', '59', '27', '20', '48', '23', '9', '26', '87', '80']


In [44]:
reranking = advanced_method(es)

TypeError: advanced_method() takes 0 positional arguments but 1 was given